In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.transducer as transducer
import tensorflow as tf






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




In [4]:
config = malaya_speech.config.transducer_featurizer_config
featurizer = malaya_speech.utils.tf_featurization.STTFeaturizer(**config)

In [5]:
k = 0.0062169171196134855

i = tf.placeholder(tf.float32, [None, None])
lengths = tf.placeholder(tf.float32, [None])
v = tf.expand_dims(tf.map_fn(featurizer.vectorize, i), -1)
length_v = tf.cast(lengths * k, tf.int32)

In [6]:
model = transducer.streaming.Model(vocabulary_size = 100)

In [7]:
p = tf.placeholder(tf.int32, [None, None])

In [8]:
logits = model([v, p], training = True)
logits

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


<tf.Tensor 'StreamingTransducer/StreamingTransducer_joint/StreamingTransducer_joint_vocab/BiasAdd:0' shape=(?, ?, ?, 100) dtype=float32>

In [9]:
greedy_decoded = model.greedy_decoder(v)

In [10]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [11]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [12]:
padded, lens = malaya_speech.padding.sequence_1d([y, y1], return_len = True)
t = [[1,1,1,1,1,1], [1,1,1,1,1,1]]

In [13]:
sess.run(logits, feed_dict = {i: padded, lengths: lens, p: t}).shape

(2, 94, 6, 100)

In [15]:
# %%time

# greedy = sess.run(greedy_decoded, feed_dict = {i: padded})
# greedy.shape